In [14]:
import pandas as pd
import os

file_path = "data/orders.csv"
df = None

if not os.path.exists(file_path):
    print(f"❌ File does not exist: {file_path}")
else:
    try:
        df = pd.read_csv(file_path, na_values=["unknown", "Not Available"]) # Add new words in the list of missing and null values.
        
        if df.empty:
            print("⚠️ CSV file is empty")
        else:
            print(f"✓ Successfully loaded {df.shape[0]} rows × {df.shape[1]} columns")
    except Exception as e:
        print(f"❌ Error loading CSV: {type(e).__name__}: {e}")

✓ Successfully loaded 9994 rows × 16 columns


In [15]:
## Get basic info about the dataset
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Order Id          9994 non-null   int64 
 1   Order Date        9994 non-null   object
 2   Ship Mode         9988 non-null   object
 3   Segment           9994 non-null   object
 4   Country           9994 non-null   object
 5   City              9994 non-null   object
 6   State             9994 non-null   object
 7   Postal Code       9994 non-null   int64 
 8   Region            9994 non-null   object
 9   Category          9994 non-null   object
 10  Sub Category      9994 non-null   object
 11  Product Id        9994 non-null   object
 12  cost price        9994 non-null   int64 
 13  List Price        9994 non-null   int64 
 14  Quantity          9994 non-null   int64 
 15  Discount Percent  9994 non-null   int64 
dtypes: int64(6), object(10)
memory usage: 1.2+ MB
None


In [16]:
# look at the first 3 rows
print("\n ## --- First 3 rows ---\n", df.head(3))

# look at the last 3 rows
print("\n ## --- last 3 rows ---\n", df.tail(3))

# look at random 3 rows
print("\n ## --- random 3 rows ---\n", df.sample(n=3))


 ## --- First 3 rows ---
    Order Id  Order Date     Ship Mode    Segment        Country         City  \
0         1  2023-03-01  Second Class   Consumer  United States    Henderson   
1         2  2023-08-15  Second Class   Consumer  United States    Henderson   
2         3  2023-01-10  Second Class  Corporate  United States  Los Angeles   

        State  Postal Code Region         Category Sub Category  \
0    Kentucky        42420  South        Furniture    Bookcases   
1    Kentucky        42420  South        Furniture       Chairs   
2  California        90036   West  Office Supplies       Labels   

        Product Id  cost price  List Price  Quantity  Discount Percent  
0  FUR-BO-10001798         240         260         2                 2  
1  FUR-CH-10000454         600         730         3                 3  
2  OFF-LA-10000240          10          10         2                 5  

 ## --- last 3 rows ---
       Order Id  Order Date       Ship Mode   Segment        Count

In [17]:
#Check for Data Quality Issues

# Check for missing values
print("==> Check for missing values : \n", df.isnull().sum(), sep="\n")

# Check the values
df['Ship Mode'].unique()

# Check for duplicate rows
print("\n===> Check for duplicate rows : \n", df.duplicated().sum(), sep="\n")

==> Check for missing values : 

Order Id            0
Order Date          0
Ship Mode           6
Segment             0
Country             0
City                0
State               0
Postal Code         0
Region              0
Category            0
Sub Category        0
Product Id          0
cost price          0
List Price          0
Quantity            0
Discount Percent    0
dtype: int64

===> Check for duplicate rows : 

0


In [18]:
# Rename columns name
print("Columns name Befor -->\n", df.columns)
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
print("Columns name After -->\n", df.columns)

Columns name Befor -->
 Index(['Order Id', 'Order Date', 'Ship Mode', 'Segment', 'Country', 'City',
       'State', 'Postal Code', 'Region', 'Category', 'Sub Category',
       'Product Id', 'cost price', 'List Price', 'Quantity',
       'Discount Percent'],
      dtype='object')
Columns name After -->
 Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')


In [19]:
# Derive new columns discount, sale price and profit
df['discount'] = df['list_price'] * df['discount_percent'] * 0.01
df['sale_price'] = df['list_price'] - df['discount']
df['profit'] = df['sale_price'] - df['cost_price']

In [20]:
# convert order date from object data type to date data type
df['order_date'] = pd.to_datetime(df['order_date'], format="%Y-%m-%d")

In [21]:
# Remove unused columns
df.drop(columns=["cost_price", "list_price", "discount_percent"], inplace=True)

In [22]:
import psycopg2
from psycopg2 import OperationalError

In [ ]:
# Connect to Postgresql database to create sales database
try:
    conn = psycopg2.connect(
        host="localhost",
        database="postgres",
        user="postgres",
        password="P@ssw0rd",
        port=5432
    )
    
    if conn.status == psycopg2.extensions.STATUS_READY:
        print("✓ Connection is ready")
        conn.autocommit = True
        cursor = conn.cursor()
        try:
            cursor.execute("CREATE DATABASE sales")
            print("✓ Database 'sales' created successfully")
        except psycopg2.errors.DuplicateDatabase:
            print("✓ Database 'sales' already exists")
        cursor.close()
        conn.close()
        
except Exception as e:
    print(f"❌ Connection Error: {e}")

✓ Connection is ready
✓ Database 'sales' created successfully


In [12]:
# Connect to postgresql server
try:
    conn = psycopg2.connect(
        database="sales",
        host="localhost",
        user="postgres",
        password="P@ssw0rd",
        port="5432"
    )
    if conn.status == psycopg2.extensions.STATUS_READY:
        print("✓ Connection is ready")
        cursor = conn.cursor()
        cursor.execute("SELECT current_database(), current_user;")
        db_name, user = cursor.fetchone()
        print(f"✓ Connected to database: {db_name}")
        print(f"✓ Connected as user: {user}")
except OperationalError as e:
    print(f"❌ Operational Error: {e}")
except Exception as e:
    print(f"❌ Connection Error: {e}")

✓ Connection is ready
✓ Connected to database: sales
✓ Connected as user: postgres


In [23]:
# Load the data into the data base
import io

table_name = 'orders'
qry = """
        CREATE TABLE orders (
        order_id INTEGER PRIMARY KEY,
        order_date DATE,
        ship_mode VARCHAR(20),
        segment VARCHAR(20),
        country VARCHAR(20),
        city VARCHAR(20),
        state VARCHAR(20),
        region VARCHAR(20),
        category VARCHAR(20),
        sub_category VARCHAR(20),
        product_id VARCHAR(20),
        discount REAL,
        sale_price REAL,
        profit REAL,
        quantity INTEGER,
        postal_code INTEGER);
    """

try:
    # Convert DataFrame to CSV in memory
    output = io.StringIO()
    df.to_csv(output, sep='\t', header=False, index=False, na_rep='\\N')
    output.seek(0)
    # Use COPY to load data
    cursor.execute(qry)
    cursor.copy_from(
            output, 
            table_name,
            columns=df.columns.tolist(),
            sep='\t',
            null='NULL'
        )
    conn.commit()
    print(f"✓ load {len(df)} rows into <Sales> Data Base in <orders> Table")
except Exception as e:
    conn.rollback()
    print(f"❌ load failed: {e}")

✓ load 9994 rows into <Sales> Data Base in <orders> Table


In [ ]:
def exec_show_qry(query: str):
    try:
        if conn.status != psycopg2.extensions.STATUS_READY:
            conn.rollback()
        cursor.execute(qry)
        result_qry = cursor.fetchall()
        print(f"Query returned {len(result_qry)} rows:")
        for i, row in enumerate(result_qry, 1):
            print(f"{i}: {row}")
    except psycopg2.Error as e:
        conn.rollback()
        print(f"❌ Database error: {e}")
    except Exception as e:
        conn.rollback()
        print(f"❌ Error failed To Exec This Query: {e}")

In [25]:
# What is The top 10 highest reveue generating products ?
qry = """
        SELECT product_id, SUM(sale_price) as sales
        FROM public.orders
        GROUP BY product_id
        ORDER BY sales DESC
        LIMIT 10;
    """
exec_show_qry(qry)

Query returned 10 rows:
1: ('TEC-CO-10004722', 59514.0)
2: ('OFF-BI-10003527', 26525.3)
3: ('TEC-MA-10002412', 21734.4)
4: ('FUR-CH-10002024', 21096.2)
5: ('OFF-BI-10001359', 19090.2)
6: ('OFF-BI-10000545', 18249.0)
7: ('TEC-CO-10001449', 18151.2)
8: ('TEC-MA-10001127', 17906.4)
9: ('OFF-BI-10004995', 17354.8)
10: ('OFF-SU-10000151', 16325.8)


In [ ]:
# What's the 5 highest selling products of each region ?
qry = """
        with ranked_products as (
            select 
                region, product_id, 
                SUM(sale_price) as sale,
                rank() over(partition by region order by SUM(sale_price) DESC) as product_rank
            from public.orders
            group by region, product_id
        )
        SELECT *
        FROM ranked_products
        WHERE product_rank <= 5
        order by region;
    """
exec_show_qry(query=qry)

In [ ]:
# Find month over month growth comparaison for 2022 and 2023 sales eg: jan 2022 vs jan 2023
qry = """
        WITH cte as (
            SELECT 
                EXTRACT(YEAR FROM order_date)::INTEGER as order_year,
                EXTRACT(MONTH FROM order_date)::INTEGER as order_month,
                SUM(sale_price) as total_sales
            FROM public.orders
            GROUP BY order_year, order_month
        )
        SELECT 
            order_month,
            SUM(case when order_year=2022 then total_sales else 0 end) as sales_2022,
            SUM(case when order_year=2023 then total_sales else 0 end) sales_2022
        FROM cte
        GROUP BY order_month
        ORDER BY order_month;
    """
exec_show_qry(qry)

In [ ]:
# For each category which month had highest sales
qry="""
        with cte as (
            SELECT 
                category,
                EXTRACT(YEAR FROM order_date)::INTEGER as year,
                EXTRACT(MONTH FROM order_date)::INTEGER as month,
                SUM(sale_price) as sale,
                ROW_NUMBER() OVER(partition by category order by SUM(sale_price) DESC) as sale_rank
            FROM public.orders
            GROUP BY category, month, year
            ORDER BY category, sale
        )
        select category, year, month, sale
        FROM cte
        where sale_rank = 1
    """
exec_show_qry(qry)

In [ ]:
# Which sub category had highest growth by profit in 2023 compare to 2022
qry="""
        with cte as (
            SELECT 
                sub_category,
                EXTRACT(YEAR FROM order_date)::INTEGER as year,
                SUM(profit) as income
            FROM public.orders
            GROUP BY sub_category, year
            ORDER BY sub_category, income
        )
        select 
            sub_category,
            MAX(CASE WHEN year = 2023 THEN income END) - MAX(CASE WHEN year = 2022 THEN income END) AS difference,
            ((MAX(CASE WHEN year = 2023 THEN income END) - MAX(CASE WHEN year = 2022 THEN income END)) * 100) / MAX(CASE WHEN year = 2022 THEN income END)  AS rate_growth
        FROM cte
        GROUP BY sub_category
        ORDER BY difference DESC
        LIMIT 1
    """
exec_show_qry(qry)